In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import soundfile as sf
from spleeter.separator import Separator

def plot_spleeter_results(input_path, output_dir='output', sr=16000, n_fft=2048, hop_length=512, compare_with_original=False, original_path=None):
    # Đọc file đầu vào
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"File {input_path} không tồn tại")
    input_audio, sr = librosa.load(input_path, sr=sr, mono=True)
    
    # Chạy Spleeter
    separator = Separator('spleeter:2stems-16kHz')
    separator.separate_to_file(input_path, output_dir)
    
    # Đọc file vocal và accompaniment
    vocal_path = os.path.join(output_dir, '05_final_audio/vocals.wav')
    accomp_path = os.path.join(output_dir, '05_final_audio/accompaniment.wav')
    if not os.path.exists(vocal_path) or not os.path.exists(accomp_path):
        raise FileNotFoundError("File vocals.wav hoặc accompaniment.wav không tồn tại")
    
    vocal_audio, _ = librosa.load(vocal_path, sr=sr, mono=True)
    accomp_audio, _ = librosa.load(accomp_path, sr=sr, mono=True)
    
    # Tính STFT
    input_stft = librosa.stft(input_audio, n_fft=n_fft, hop_length=hop_length, window='hann')
    vocal_stft = librosa.stft(vocal_audio, n_fft=n_fft, hop_length=hop_length, window='hann')
    accomp_stft = librosa.stft(accomp_audio, n_fft=n_fft, hop_length=hop_length, window='hann')
    
    # Vẽ spectrogram so sánh
    plt.figure(figsize=(15, 12))
    
    plt.subplot(3, 1, 1)
    S_input_db = librosa.amplitude_to_db(np.abs(input_stft))
    librosa.display.specshow(S_input_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram - Tín hiệu đầu vào (Sau Phase 1)')
    
    plt.subplot(3, 1, 2)
    S_vocal_db = librosa.amplitude_to_db(np.abs(vocal_stft))
    librosa.display.specshow(S_vocal_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram - Vocal (Spleeter)')
    
    plt.subplot(3, 1, 3)
    S_accomp_db = librosa.amplitude_to_db(np.abs(accomp_stft))
    librosa.display.specshow(S_accomp_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram - Accompaniment (Spleeter)')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'spleeter_spectrograms.png'))
    
    # Vẽ dạng sóng so sánh
    plt.figure(figsize=(15, 12))
    
    plt.subplot(3, 1, 1)
    librosa.display.waveshow(input_audio, sr=sr)
    plt.title('Dạng sóng - Tín hiệu đầu vào (Sau Phase 1)')
    plt.xlabel('Thời gian (s)')
    plt.ylabel('Biên độ')
    
    plt.subplot(3, 1, 2)
    librosa.display.waveshow(vocal_audio, sr=sr)
    plt.title('Dạng sóng - Vocal (Spleeter)')
    plt.xlabel('Thời gian (s)')
    plt.ylabel('Biên độ')
    
    plt.subplot(3, 1, 3)
    librosa.display.waveshow(accomp_audio, sr=sr)
    plt.title('Dạng sóng - Accompaniment (Spleeter)')
    plt.xlabel('Thời gian (s)')
    plt.ylabel('Biên độ')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'spleeter_waveforms.png'))
    
    # So sánh với tách trực tiếp từ file gốc (nếu yêu cầu)
    if compare_with_original and original_path:
        if not os.path.exists(original_path):
            raise FileNotFoundError(f"File {original_path} không tồn tại")
        
        # Chạy Spleeter trên file gốc
        separator.separate_to_file(original_path, output_dir)
        orig_vocal_path = os.path.join(output_dir, '00_original_extracted_from_video/vocals.wav')
        if not os.path.exists(orig_vocal_path):
            raise FileNotFoundError("File vocals.wav từ file gốc không tồn tại")
        
        orig_vocal_audio, _ = librosa.load(orig_vocal_path, sr=sr, mono=True)
        orig_vocal_stft = librosa.stft(orig_vocal_audio, n_fft=n_fft, hop_length=hop_length, window='hann')
        
        # Vẽ spectrogram so sánh vocal
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 1, 1)
        librosa.display.specshow(S_vocal_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram - Vocal từ Tín hiệu sau Phase 1 (Spleeter)')
        
        plt.subplot(2, 1, 2)
        S_orig_vocal_db = librosa.amplitude_to_db(np.abs(orig_vocal_stft))
        librosa.display.specshow(S_orig_vocal_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram - Vocal từ Tín hiệu gốc (Không qua Phase 1)')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'spleeter_vocal_comparison.png'))

# Thực thi
input_path = ('20250725_111327_05_preprocessed_for_ai.wav')
original_path = ('20250725_111327_00_original_extracted_from_video.wav')
plot_spleeter_results(input_path, output_dir, compare_with_original=True, original_path=original_path)

FileNotFoundError: File output\20250725_111327_05_preprocessed_for_ai.wav không tồn tại